In [41]:
%pip install pandas numpy yfinance sec-api

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
import datetime as dt
from sec_api import MappingApi

In [3]:
mappingApi = MappingApi(api_key='729dbe3ac00b681b512faf8ae965cf9adad767a811bb37187fe3fa0f7a3805b0')

df = pd.read_csv("Warren_Buffett_final_df.csv")
df = df.head(25)
print(df.head())
# print(df.size)

def cusip_to_ticker(cusip):
    result = mappingApi.resolve('cusip', cusip)
    return result[0].get('ticker')

df['ticker'] = df['cusip'].apply(cusip_to_ticker)
print(df.head())

# by_cusip = mappingApi.resolve('cusip', '88160R101')
# print(by_cusip[0].get('ticker'))

          issuer title      cusip      value    shares        date
0  ALLY FINL INC   COM  02005N100  504589507  12719675  2024-08-14
1  ALLY FINL INC   COM  02005N100  111229721   2803875  2024-08-14
2  ALLY FINL INC   COM  02005N100  167732694   4228200  2024-08-14
3  ALLY FINL INC   COM  02005N100  124444790   3137000  2024-08-14
4  ALLY FINL INC   COM  02005N100  191854038   4836250  2024-08-14
          issuer title      cusip      value    shares        date ticker
0  ALLY FINL INC   COM  02005N100  504589507  12719675  2024-08-14   ALLY
1  ALLY FINL INC   COM  02005N100  111229721   2803875  2024-08-14   ALLY
2  ALLY FINL INC   COM  02005N100  167732694   4228200  2024-08-14   ALLY
3  ALLY FINL INC   COM  02005N100  124444790   3137000  2024-08-14   ALLY
4  ALLY FINL INC   COM  02005N100  191854038   4836250  2024-08-14   ALLY


In [33]:
df = pd.read_csv('Warren_Buffett_final_df_with_ticker.csv')

tickers = list(df['ticker'].dropna().unique())
print(tickers)
# print(len(tickers))

# History Data
def get_history_data(ticker):
    stock = yf.Ticker(ticker)
    hist = stock.history(period="2y", interval="3mo")
    return hist

history_dfs = []
for ticker in tickers:
    hist = get_history_data(ticker)
    hist.index = pd.MultiIndex.from_product([[ticker], hist.index], names=["ticker", "date"])
    history_dfs.append(hist)

history_df = pd.concat(history_dfs)

history_df = history_df.reset_index()
history_df.to_csv("history_data.csv", index=False)

history_df = history_df.reset_index()
def remove_timezone_from_df(df):
    # Remove timezone from columns with datetime dtype
    for col in df.select_dtypes(include=['datetimetz']).columns:
        df[col] = df[col].dt.tz_localize(None)
    # Remove timezone from datetime index if present
    if isinstance(df.index, pd.DatetimeIndex) and df.index.tz is not None:
        df.index = df.index.tz_localize(None)
    return df

history_df = remove_timezone_from_df(history_df)

['AAPL', 'ALLY', 'AMZN', 'AON', 'AXP', 'BAC', 'BATRK', 'C', 'CB', 'CHTR', 'COF', 'CVX', 'DEO', 'DPZ', 'DVA', 'FND', 'FWONK', 'HEI.A', 'JEF', 'KHC', 'KO', 'KR', 'LEN.B', 'LILA', 'LILAK', 'LLYVA', 'LLYVK', 'LPX', 'LSXMA', 'LSXMK', 'MA', 'MCO', 'NU', 'NVR', 'OXY', 'POOL', 'SIRI', 'SPY', 'STZ', 'TMUS', 'ULTA', 'V', 'VOO', 'VRSN']


$HEI.A: possibly delisted; no price data found  (period=2y) (Yahoo error = "No data found, symbol may be delisted")
$LEN.B: possibly delisted; no price data found  (period=2y)
$LSXMA: possibly delisted; no price data found  (period=2y) (Yahoo error = "No data found, symbol may be delisted")
$LSXMK: possibly delisted; no price data found  (period=2y) (Yahoo error = "No data found, symbol may be delisted")


In [11]:
# Data Preparation (Financials, Balance Sheets, Cashflow)

tickers = list(df['ticker'].dropna().unique())
print(tickers)

# wanna apply ffill to the data
def quarterly_financials(ticker):
    stock = yf.Ticker(ticker)
    fin = stock.quarterly_financials
    return fin

def quarterly_balance_sheet(ticker):
    stock = yf.Ticker(ticker)
    bal = stock.quarterly_balance_sheet
    return bal

def quarterly_cashflow(ticker):
    stock = yf.Ticker(ticker)
    cf = stock.quarterly_cashflow
    return cf


financials_dfs = []
for ticker in tickers:
    fin = quarterly_financials(ticker).T 
    fin.index = pd.MultiIndex.from_product([[ticker], fin.index], names=["ticker", "date"])
    financials_dfs.append(fin)

balance_sheets_dfs = []
for ticker in tickers:
    bal = quarterly_balance_sheet(ticker).T
    bal.index = pd.MultiIndex.from_product([[ticker], bal.index], names=["ticker", "date"])
    balance_sheets_dfs.append(bal)

cashflow_dfs = []
for ticker in tickers:
    cf = quarterly_cashflow(ticker).T
    cf.index = pd.MultiIndex.from_product([[ticker], cf.index], names=["ticker", "date"])
    cashflow_dfs.append(cf)


financials_df = pd.concat(financials_dfs)
balance_sheets_df = pd.concat(balance_sheets_dfs)
cashflow_df = pd.concat(cashflow_dfs)

financials_df = financials_df.groupby(level=0).ffill()
balance_sheets_df = balance_sheets_df.groupby(level=0).ffill()
cashflow_df = cashflow_df.groupby(level=0).ffill()

financials_df.to_csv("quarterly_financials.csv")
balance_sheets_df.to_csv("quarterly_balance_sheets.csv")
cashflow_df.to_csv("quarterly_cashflow.csv")


['AAPL', 'ALLY', 'AMZN', 'AON', 'AXP', 'BAC', 'BATRK', 'C', 'CB', 'CHTR', 'COF', 'CVX', 'DEO', 'DPZ', 'DVA', 'FND', 'FWONK', 'HEI.A', 'JEF', 'KHC', 'KO', 'KR', 'LEN.B', 'LILA', 'LILAK', 'LLYVA', 'LLYVK', 'LPX', 'LSXMA', 'LSXMK', 'MA', 'MCO', 'NU', 'NVR', 'OXY', 'POOL', 'SIRI', 'SPY', 'STZ', 'TMUS', 'ULTA', 'V', 'VOO', 'VRSN']


In [15]:
# Profitability: ROE, profit margins, operating margins


net_income = financials_df["Net Income"]
total_revenue = financials_df["Total Revenue"]
operating_income = financials_df["Operating Income"]
equity = balance_sheets_df["Common Stock Equity"]
total_assets = balance_sheets_df["Total Assets"]
gross_profit = financials_df["Gross Profit"]

roe = net_income / equity
profit_margin = net_income / total_revenue
operating_margin = operating_income / total_revenue
net_margin = net_income / total_revenue
roa = net_income / total_assets
gross_margins = gross_profit / total_revenue


profitability_df = pd.DataFrame({
    "ROE": roe,
    "Profit Margin": profit_margin,
    "Operating Margin": operating_margin,
    "Net Margin": net_margin,
    "ROA": roa,
    "Gross Margin": gross_margins
})

print(profitability_df)


                        ROE  Profit Margin  Operating Margin  Net Margin  \
ticker date                                                                
AAPL   2023-12-31       NaN            NaN               NaN         NaN   
       2024-06-30  0.321521       0.250044          0.295557    0.250044   
       2024-09-30  0.258753       0.155230          0.311714    0.155230   
       2024-12-31  0.544204       0.292277          0.344586    0.292277   
       2025-03-31  0.370980       0.259860          0.310291    0.259860   
...                     ...            ...               ...         ...   
VRSN   2024-06-30 -0.109447       0.513562          0.687678    0.513562   
       2024-09-30 -0.105914       0.515361          0.689452    0.515361   
       2024-12-31 -0.097809       0.484320          0.667172    0.484320   
       2025-03-31 -0.100809       0.495401          0.674124    0.495401   
       2025-06-30 -0.104022       0.505977          0.684801    0.505977   

           

In [16]:
# Growth - 
def revenue_growth(financials_df):
    revenue = financials_df["Total Revenue"]
    revenue = revenue.sort_index()
    return revenue.groupby(level=0).pct_change()

def earnings_growth(financials_df):
    earnings = financials_df["Net Income"]
    earnings = earnings.sort_index()
    return earnings.groupby(level=0).pct_change()

revenue_growth_df = revenue_growth(financials_df)
# print(revenue_growth_df)

earnings_growth_df = earnings_growth(financials_df)
# print(earnings_growth_df)


growth_df = pd.DataFrame({
    "Revenue Growth": revenue_growth_df,
    "Earnings Growth": earnings_growth_df
})

print(growth_df)

                   Revenue Growth  Earnings Growth
ticker date                                       
AAPL   2024-06-30             NaN              NaN
       2024-09-30        0.106707        -0.312943
       2024-12-31        0.309386         1.465391
       2025-03-31       -0.232832        -0.317919
       2025-06-30       -0.013874        -0.054318
...                           ...              ...
VRSN   2024-06-30             NaN              NaN
       2024-09-30        0.009042         0.012575
       2024-12-31        0.012289        -0.048684
       2025-03-31        0.017451         0.040731
       2025-06-30        0.018891         0.040642

[238 rows x 2 columns]


In [20]:
# Financial Health

debt_ratio = balance_sheets_df["Total Debt"] / balance_sheets_df["Total Assets"]
# print(debt_ratio)

debt_to_equity_ratio = balance_sheets_df["Total Debt"] / balance_sheets_df["Stockholders Equity"]
# print(debt_to_equity_ratio)

current_ratio = balance_sheets_df["Current Assets"] / balance_sheets_df["Current Liabilities"]
# print(current_ratio)

FCF = cashflow_df["Free Cash Flow"]
# print(FCF)

Financial_health_df = pd.DataFrame({
    "Debt Ratio": debt_ratio,
    "Debt to Equity Ratio": debt_to_equity_ratio,
    "Current Ratio": current_ratio,
    "Free Cash Flow": FCF
})

print(Financial_health_df)

                   Debt Ratio  Debt to Equity Ratio  Current Ratio  \
ticker date                                                          
AAPL   2023-12-31    0.305490              1.518618       0.952980   
       2024-03-31         NaN                   NaN            NaN   
       2024-06-30    0.305490              1.518618       0.952980   
       2024-09-30    0.292150              1.872327       0.867313   
       2024-12-31    0.281323              1.449999       0.922938   
...                       ...                   ...            ...   
VRSN   2024-06-30    1.197861             -0.992568       0.638966   
       2024-09-30    1.232627             -0.948174       0.470056   
       2024-12-31    1.280839             -0.920118       0.433545   
       2025-03-31    1.237618             -0.906272       0.555029   
       2025-06-30    1.272624             -0.898586       0.541680   

                   Free Cash Flow  
ticker date                        
AAPL   2023-12-31

In [34]:
# Price & Return Metrics (per quarter):
# history_df = history_df.reset_index()
history_df["date"] = pd.to_datetime(history_df["date"])
history_df = history_df.set_index("date")

quarterly = history_df.groupby("ticker")["Close"].resample("QE").ffill().pct_change()
print(quarterly)

volatility = history_df.groupby("ticker")["Close"].resample("QE").ffill().pct_change().groupby("ticker").std()
print(volatility)


ticker  date      
AAPL    2023-09-30         NaN
        2023-12-31    0.081273
        2024-03-31   -0.075084
        2024-06-30    0.305486
        2024-09-30    0.018626
                        ...   
VRSN    2024-09-30   -0.054382
        2024-12-31    0.215788
        2025-03-31    0.312186
        2025-06-30   -0.046966
        2025-09-30    0.006531
Name: Close, Length: 360, dtype: float64
ticker
AAPL       0.128051
ALLY       0.394730
AMZN       0.796014
AON        0.121862
AXP        0.279659
BAC        0.356807
BATRK      0.135377
C          0.218624
CB         0.402522
CHTR       0.292366
COF        0.283984
CVX        0.149606
DEO        0.071610
DPZ        0.657080
DVA        0.359829
FND        0.197819
FWONK      0.129250
JEF        0.346150
KHC        0.195743
KO         0.301683
KR         0.171116
LILA       0.386258
LILAK      0.233007
LLYVA      0.933462
LLYVK      0.310056
LPX        0.259197
MA         0.895253
MCO        0.223953
NU         0.373362
NVR      137

In [19]:
# Valuation

def get_info_data(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    return info

info_dfs = []
for ticker in tickers:
    info = get_info_data(ticker)
    info_df = pd.DataFrame([info])
    info_df["ticker"] = ticker
    info_dfs.append(info_df)

info_df = pd.concat(info_dfs, ignore_index=True)
# print(info_df)

# Trailing P/E Ratio
trailing_pe = info_df["trailingPE"]
# print(info_df[["ticker", "trailingPE"]])

# priceToBook
price_to_book = info_df["priceToBook"]
# print(info_df[["ticker", "priceToBook"]])

# enterpriseToEbitda (EV/EBITDA)
enterprise_to_ebitda = info_df["enterpriseToEbitda"]
# print(info_df[["ticker", "enterpriseToEbitda"]])

# enterpriseValue (EV)
enterprise_value = info_df["enterpriseValue"]
# print(info_df[["ticker", "enterpriseValue"]])

# marketCap
market_cap = info_df["marketCap"]
# print(info_df[["ticker", "marketCap"]])

# dividendYield
dividend_yield = info_df["dividendYield"]
# print(info_df[["ticker", "dividendYield"]])

fundamental_df = pd.DataFrame({
    "ticker": info_df["ticker"],
    "trailingPE": trailing_pe,
    "priceToBook": price_to_book,
    "enterpriseToEbitda": enterprise_to_ebitda,
    "enterpriseValue": enterprise_value,
    "marketCap": market_cap,
    "dividendYield": dividend_yield
})

print(fundamental_df)


   ticker  trailingPE  priceToBook  enterpriseToEbitda  enterpriseValue  \
0    AAPL   35.089394    52.265850              24.582     3.483212e+12   
1    ALLY   37.355770     0.978269                 NaN     2.351934e+10   
2    AMZN   35.217987     7.378557              18.907     2.530305e+12   
3     AON   30.971308    10.093507              18.135     9.633212e+10   
4     AXP   21.466621     6.580002                 NaN     2.155321e+11   
5     BAC   13.765395     1.264309                 NaN     3.523061e+11   
6   BATRK         NaN     5.273076              73.542     3.487857e+09   
7       C   13.838996     0.876074                 NaN    -2.246928e+10   
8      CB   12.106401     1.575286              11.318     1.292529e+11   
9    CHTR    7.330961     2.256716               6.152     1.364151e+11   
10    COF  582.243200     1.305280                 NaN     1.380342e+11   
11    CVX   20.173970     1.847568               8.105     2.967589e+11   
12    DEO   26.229313    

C:\Users\revan\AppData\Local\Temp\ipykernel_24796\291642618.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  info_df = pd.concat(info_dfs, ignore_index=True)


In [22]:
%pip install openpyxl


   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpy

In [35]:
# history_df
# financials_df
# balance_sheets_df
# cashflow_df
# profitability_df
# growth_df
# Financial_health_df
# quarterly
# Volatility
# fundamental_df

output_path = "Warren_Buffett_Analysis.xlsx"

with pd.ExcelWriter(output_path) as writer:
    history_df.to_excel(writer, sheet_name="Price_History")
    financials_df.to_excel(writer, sheet_name="Financials")
    balance_sheets_df.to_excel(writer, sheet_name="Balance_Sheets")
    cashflow_df.to_excel(writer, sheet_name="Cashflow")
    profitability_df.to_excel(writer, sheet_name="Profitability")
    growth_df.to_excel(writer, sheet_name="Growth")
    Financial_health_df.to_excel(writer, sheet_name="Financial_Health")
    quarterly.to_excel(writer, sheet_name="Quarterly_Returns")
    volatility.to_excel(writer, sheet_name="Volatility")
    fundamental_df.to_excel(writer, sheet_name="Valuation")

print(f"All DataFrames have been successfully written to '{output_path}'")

All DataFrames have been successfully written to 'Warren_Buffett_Analysis.xlsx'
